### Recommend System

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio

In [118]:
train_data = scio.loadmat("./ex8_movies.mat")
print(train_data.keys())
Y = train_data['Y']
R = train_data['R']
print(Y.shape, R.shape)

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'R'])
(1682, 943) (1682, 943)


In [4]:
avg = np.mean(Y, axis=1)
print(avg[0])

1.8589607635206786


In [128]:
def get_loss(X, Theta, Y, R, lamb=0):
    return np.sum(((X.dot(Theta.T) - Y) * R) ** 2) / 2 + lamb * np.sum(Theta ** 2) / 2 + lamb * np.sum(X ** 2) / 2

def grad_by_x(X, Theta, Y, R, lamb=0):
    return ((X.dot(Theta.T) - Y) * R).dot(Theta) + lamb * X

def grad_by_theta(X, Theta, Y, R, lamb=0):
    return ((X.dot(Theta.T) - Y) * R).T.dot(X) + lamb * Theta


In [167]:
train_data = scio.loadmat("./ex8_movieParams.mat")
print(train_data.keys())
X_d = train_data['X']
Theta_d = train_data['Theta']
print(X_d.shape, Theta_d.shape)
print(get_loss(X_d[:5,:3], Theta_d[:4,:3], Y[:5,:4], R[:5,:4]))
print(get_loss(X_d[:5,:3], Theta_d[:4,:3], Y[:5,:4], R[:5,:4], lamb=1.5))

print(get_loss(X_d, Theta_d, Y, R))
print(get_loss(X_d, Theta_d, Y, R, 1))

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])
(1682, 10) (943, 10)
22.224603725685675
31.34405624427422
27918.64012454421
32520.682450229557


In [126]:
epsilon = 1e-5
X_d_plus = np.copy(X_d)
X_d_plus[0][0] += epsilon
X_d_minus = np.copy(X_d)
X_d_minus[0][0] -= epsilon

grad_expected = (get_loss(X_d_plus, Theta_d, Y, R) - get_loss(X_d_minus, Theta_d, Y, R)) / (2 * epsilon)
print(grad_expected)
grad = grad_by_x(X_d, Theta_d, Y, R)
print(grad[0][0])
diff = np.linalg.norm(grad_expected - grad[0][0]) / np.linalg.norm(grad_expected + grad[0][0])

print(diff)

-6.261841372179332
-6.261841441787519
5.5581243795574405e-09


In [180]:
def train(Y, X, Theta, R, lr=1e-4, epochs=1000, lamb=1):
    for e in range(epochs):
        if e % 100 == 0:
            loss = get_loss(X, Theta, Y, R, lamb)
            print('loss:', loss)
        X -= lr * grad_by_x(X, Theta, Y, R, lamb)
        Theta -= lr * grad_by_theta(X, Theta, Y, R, lamb)
#     loss = get_loss(X, Theta, Y, R, lamb)
#     print(loss)
    return X, Theta

In [153]:
N = 10
num_movie, num_user = Y.shape
X = np.random.rand(num_movie, N)
Theta = np.random.rand(num_user, N)
X, Theta = train(Y, X, Theta, R)
predict = X.dot(Theta.T)
print(predict[:,1])

[2.59488072 2.28244827 2.0749319  ... 1.52561349 2.99964275 2.19826318]


In [123]:
import codecs

f = codecs.open("movie_ids.txt", "r", encoding = "ISO-8859-1")
lines = f.readlines()
movie_ids = {}
for line in lines:
    i = line.index(' ')
    movie_ids[int(line[:i])] = line[i:]
# print(movie_ids)

my_ratings = np.zeros([len(Y)])
my_ratings[0] = 4
my_ratings[97] = 2
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63]= 5
my_ratings[65]= 3
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354]= 5

print("New user ratings:\n")
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print("Rated {} for {}".format(my_ratings[i], movie_ids[i + 1]))
        

New user ratings:
Rated 4.0 for  Toy Story (1995)

Rated 3.0 for  Twelve Monkeys (1995)

Rated 5.0 for  Usual Suspects, The (1995)

Rated 4.0 for  Outbreak (1995)

Rated 5.0 for  Shawshank Redemption, The (1994)

Rated 3.0 for  While You Were Sleeping (1995)

Rated 5.0 for  Forrest Gump (1994)

Rated 2.0 for  Silence of the Lambs, The (1991)

Rated 4.0 for  Alien (1979)

Rated 5.0 for  Die Hard 2 (1990)

Rated 5.0 for  Sphere (1998)



In [187]:
num_feat = 50
Ynew = np.c_[my_ratings, Y]
Rnew = np.c_[np.array([1 if r > 0 else 0 for r in my_ratings]), R]
print(Ynew.shape, Rnew.shape)
Xnew = np.random.standard_normal((len(Ynew), num_feat))
Tnew = np.random.standard_normal((Ynew.shape[1], num_feat))
Ymean = np.mean(Ynew, axis=1).reshape(-1, 1)
Ynorm = Ynew - Ymean
# print(np.mean(Ynorm))
Xtrain, Ttrain = train(Ynorm, Xnew, Tnew, Rnew, lr=2e-3, epochs=500, lamb=10)
predict = Xtrain.dot(Ttrain.T)
my_predict = predict[:,0] + Ymean
print(predict[:10,0])

# indexes = sorted(range(len(predict[:,0])), key=lambda i: predict[i,0], reverse=True)
indexes = np.argsort(-predict[:,0])
# print(indexes)

print("Top recommendations for you:\n")
for i in indexes[:10]:
    print("Predicting rating %.2f for movie %s" % (predict[i,0], movie_ids[i + 1]))

(1682, 944) (1682, 944)
loss: 3640386.6105396235
loss: 115819.06503815248
loss: 92381.05829787612
loss: 96896.9651042002
loss: 92749.08955709089
[1.61448307 2.05922556 1.79214008 2.13710909 1.82494634 0.80732568
 1.35255467 2.04033561 1.41987153 1.82037749]
Top recommendations for you:
Predicting rating 6.36 for movie  Return of the Jedi (1983)
 

Predicting rating 4.17 for movie  Star Wars (1977)
 

Predicting rating 2.65 for movie  Die Hard (1988)
 

Predicting rating 2.63 for movie  Close Shave, A (1995)
 

Predicting rating 2.62 for movie  Wallace & Gromit: The Best of Aardman Animation (1996)
 

Predicting rating 2.60 for movie  Shawshank Redemption, The (1994)
 

Predicting rating 2.59 for movie  Great Escape, The (1963)
 

Predicting rating 2.59 for movie  Face/Off (1997)
 

Predicting rating 2.56 for movie  Terminator 2: Judgment Day (1991)
 

Predicting rating 2.54 for movie  Forrest Gump (1994)
 

